In [2]:
import pandas as pd
DataKey  = pd.read_csv("keyword/tmdb_5000_movies.csv")
DataKey.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [3]:
import json
def extract_genres(genre_str):
    if not genre_str or genre_str == '[]':
        return []
    try:
        genres = json.loads(genre_str)
        return [g['name'] for g in genres]
    except (json.JSONDecodeError, TypeError):
        return []

DataKey = DataKey[['title', 'genres', 'overview']]
DataKey.dropna(inplace=True)

DataKey['genres'] = DataKey['genres'].apply(extract_genres)
print(DataKey.isna().sum())
DataKey.head()


title       0
genres      0
overview    0
dtype: int64


,title,genres,overview
0,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","In the 22nd century, a paraplegic Marine is di..."
1,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","Captain Barbossa, long believed to be dead, ha..."
2,Spectre,"[Action, Adventure, Crime]",A cryptic message from Bond’s past sends him o...
3,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]",Following the death of District Attorney Harve...
4,John Carter,"[Action, Adventure, Science Fiction]","John Carter is a war-weary, former military ca..."


In [4]:
DataKey['genres'] = DataKey['genres'].apply(lambda x: " ".join(x))
DataKey['text'] = DataKey['genres'] + " " + DataKey['overview']
DataKey.head()

,title,genres,overview,text
0,Avatar,Action Adventure Fantasy Science Fiction,"In the 22nd century, a paraplegic Marine is di...",Action Adventure Fantasy Science Fiction In th...
1,Pirates of the Caribbean: At World's End,Adventure Fantasy Action,"Captain Barbossa, long believed to be dead, ha...","Adventure Fantasy Action Captain Barbossa, lon..."
2,Spectre,Action Adventure Crime,A cryptic message from Bond’s past sends him o...,Action Adventure Crime A cryptic message from ...
3,The Dark Knight Rises,Action Crime Drama Thriller,Following the death of District Attorney Harve...,Action Crime Drama Thriller Following the deat...
4,John Carter,Action Adventure Science Fiction,"John Carter is a war-weary, former military ca...",Action Adventure Science Fiction John Carter i...


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

tfidf = TfidfVectorizer(stop_words="english")
tfidf_matrix = tfidf.fit_transform(DataKey['text'])
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

def get_recommendations(title, cosine_sim=cosine_sim):
    # 1. Case-insensitive match
    matches = DataKey[DataKey['title'].str.lower() == title.lower()]
    
    # If no title found
    if matches.empty:
        return f"Title '{title}' not found."

    # 2. Get index
    idx = matches.index[0]

    # 3. Compute similarity scores
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]

    movie_indices = [i[0] for i in sim_scores]

    return DataKey['title'].iloc[movie_indices]


get_recommendations('toy story')

42                 Toy Story 3
343                Toy Story 2
1779    The 40 Year Old Virgin
2869    For Your Consideration
891            Man on the Moon
3873             Class of 1984
3379              Factory Girl
3065                Heartbeeps
3383                 Losin' It
2569               Match Point
Name: title, dtype: object

In [6]:
cols=['user_id','movie_id','rating']
DataUser = pd.read_csv('user/u.data', sep='\t', header=None, names=['user_id','movie_id','rating','timestamp'], usecols=['user_id','movie_id','rating'], encoding='latin-1')
print(DataUser.head())
print(DataUser.isna().sum())


   user_id  movie_id  rating
0      196       242       3
1      186       302       3
2       22       377       1
3      244        51       2
4      166       346       1
user_id     0
movie_id    0
rating      0
dtype: int64


In [7]:
print("unique users:", DataUser['user_id'].nunique())
print("unique movies:", DataUser['movie_id'].nunique())
print("total ratings:", len(DataUser))

unique users: 943
unique movies: 1682
total ratings: 100000


In [8]:
import re

df = pd.read_csv('user/u.item', sep='|', header=None, usecols=[0,1],
                 names=['movie_id','name'], encoding='latin-1')

def clean_name(s):
    return re.sub(r'\s*\(\d{4}\)\s*$', '', s).strip()

df['name'] = df['name'].apply(clean_name)

print(df.head())
print(df.isna().sum())

   movie_id        name
0         1   Toy Story
1         2   GoldenEye
2         3  Four Rooms
3         4  Get Shorty
4         5     Copycat
movie_id    0
name        0
dtype: int64
